In [0]:
dbutils.widgets.text("catalog", "dev")
dbutils.widgets.text("silver_schema", "02_silver")
dbutils.widgets.text("gold_schema", "03_gold_product")

In [0]:
catalog = dbutils.widgets.get("catalog")
silver_schema = dbutils.widgets.get("silver_schema")
gold_schema = dbutils.widgets.get("gold_schema")

In [0]:
# Create catalog and schema dynamically using Python variables
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{gold_schema}")

In [0]:
product_b2b_sql = f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.product_b2b AS
SELECT 
  di.invoice_id,
  di.invoice_item_no,
  di.product_id,
  di.sales_order_id,
  di.sales_order_item_no,
  di.invoiced_quantity as brought_quantity,
  di.primary_unit,
  di.invoice_item_total_amount as item_total,
  di.invoice_date,
  so.distributor_id, 
  so.order_date, 
  so.currency, 
  so.order_quantity, 
  so.unit_measure, 
  so.unit_price as selling_price, 
  p.product_name,
  p.brand,
  p.manufacturer,
  p.category,
  p.department,
  p.unit_price as cost_price,
  p.retail_price as mrp
FROM {catalog}.{silver_schema}.distributor_invoice_mv di 
LEFT JOIN {catalog}.{silver_schema}.distributor_sale_order_mv so 
  ON di.sales_order_id = so.order_id 
  AND di.sales_order_item_no = so.item_no
LEFT JOIN {catalog}.{silver_schema}.product_mv p 
  ON di.product_id = p.product_id
"""
spark.sql(product_b2b_sql)

In [0]:
product_b2c_sql = f"""
CREATE OR REPLACE VIEW {catalog}.{gold_schema}.product_b2c AS
SELECT 
  co.order_id,
  co.order_item_id,
  co.product_id,
  co.quantity as brought_quantity,
  co.unit_price as selling_price,
  co.consumer_id,
  co.order_date,
  co.order_status,
  co.total_price as item_total,
  co.currency,
  -- co.payment_method,
  p.product_name,
  p.brand,
  p.manufacturer,
  p.category,
  p.department,
  p.unit_price as cost_price,
  p.retail_price as mrp
FROM {catalog}.{silver_schema}.consumer_orders_mv co
LEFT JOIN {catalog}.{silver_schema}.product_mv p ON co.product_id = p.product_id
WHERE co.order_status = 'Delivered'
"""
spark.sql(product_b2c_sql)